## Week 2 - Pizza Runner

### <u>Introduction </u>

Did you know that over 115 million kilograms of pizza are devoured daily worldwide? This astounding statistic, coupled with Danny's serendipitous encounter with the phrase "80s Retro Styling and Pizza Is The Future!" on Instagram, sparked the inception of Pizza Runner. Recognizing that traditional pizza alone wouldn't suffice to propel his vision forward, Danny conceived the idea to "Uberize" pizza delivery—a concept that laid the foundation for Pizza Runner.

### <u>Entity-Relationship Diagram </u>
<div>
<img src="../sql-screenshots/pizza_runner_images/entity-diagram.png" width="500"/>
</div>


#### A. Pizza Metrics
##### 1. How many pizzas were ordered?
```sql
SELECT COUNT(pizza_id) AS pizza_count 
FROM pizza_runner.customer_orders
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s1.png" width="250"/>
</div>

##### 2. How many unique customer orders were made?
```sql 
SELECT COUNT(DISTINCT customer_id) AS unique_customer_orders 
FROM pizza_runner.customer_orders
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s2.png" width="250"/>
</div>

##### 3. How many successful orders were delivered by each runner?
```sql 
SELECT runner_id,
SUM(CASE 
  WHEN cancellation IS NULL OR 
       cancellation = 'null' OR 
       nullif(cancellation, '') IS NULL 
       THEN 1 ELSE 0 
  END) AS success_orders
FROM pizza_runner.runner_orders AS ro 
GROUP BY runner_id
ORDER BY runner_id 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s3.png" width="250"/>
</div>

##### 4. How many of each type of pizza was delivered?
```sql
SELECT pizza_name,
SUM(CASE 
  WHEN cancellation IS NULL OR 
       cancellation = 'null' OR 
       nullif(cancellation, '') IS NULL 
       THEN 1 ELSE 0 
  END) AS success_orders
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
LEFT JOIN pizza_runner.pizza_names AS pn 
ON co.pizza_id = pn.pizza_id 
GROUP BY pizza_name 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s4.png" width="250"/>
</div>

##### 5. How many Vegetarian and Meatlovers were ordered by each customer?


```sql
SELECT customer_id, pizza_name, COUNT(pizza_name) AS pizza_count 
FROM pizza_runner.customer_orders AS co 
LEFT JOIN pizza_runner.pizza_names AS pn 
ON co.pizza_id = pn.pizza_id
GROUP BY customer_id, pizza_name 
ORDER BY customer_id
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s5.png" width="250"/>
</div>

##### 6. What was the maximum number of pizzas delivered in a single order?


```sql 
WITH t1 AS (
SELECT ro.order_id, 
  COUNT(CASE WHEN cancellation IS NULL OR 
       cancellation = 'null' OR 
       nullif(cancellation, '') IS NULL 
       THEN 1 ELSE 0 
  END) AS pizza_success
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
GROUP BY ro.order_id 
)

SELECT order_id, MAX(pizza_success) AS max_pizza_order 
FROM t1 
GROUP BY order_id 
ORDER BY max_pizza_order DESC
LIMIT 1 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s6.png" width="250"/>
</div>

##### 7. For each customer, how many delivered pizzas had at least 1 change and how many had no changes?


```sql 
WITH t1 AS (
SELECT *, 
  CASE WHEN cancellation IS NULL 
  OR cancellation = 'null' 
  OR nullif(cancellation, '') IS NULL 
  THEN 1 ELSE 0 
  END AS cancel_status,
   CASE WHEN exclusions IS NULL 
            OR exclusions = 'null' 
            OR nullif(exclusions, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS exclusion_status, 
  CASE WHEN extras ='null' 
            OR nullif(extras, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS extras_status
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id
)

SELECT customer_id, SUM(change_count) AS total_change,SUM(no_change_count) AS total_no_change FROM 
(
  SELECT customer_id, 
      CASE WHEN exclusion_status = 1 AND extras_status = 1 
              OR exclusion_status = 1 AND extras_status = 0 
              OR exclusion_status = 0 AND extras_status = 1
              THEN 1 ELSE 0 END AS change_count,
      CASE WHEN exclusion_status = 0 AND extras_status = 0 
              THEN 1 ELSE 0 END as no_change_count
      FROM t1 
      WHERE cancel_status = 1 
)
GROUP BY customer_id 
ORDER BY customer_id 
```


<div>
<img src="../sql-screenshots/pizza_runner_images/s7.png" width="250"/>
</div>

##### 8. How many pizzas were delivered that had both exclusions and extras?


```sql 
WITH t1 AS (
SELECT *, 
  CASE WHEN cancellation IS NULL 
  OR cancellation = 'null' 
  OR nullif(cancellation, '') IS NULL 
  THEN 1 ELSE 0 
  END AS cancel_status,
   CASE WHEN exclusions IS NULL 
            OR exclusions = 'null' 
            OR nullif(exclusions, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS exclusion_status, 
  CASE WHEN extras ='null' 
            OR nullif(extras, '') IS NULL 
  THEN 0 
  ELSE 1 
  END AS extras_status
FROM pizza_runner.runner_orders AS ro 
LEFT JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id
)

SELECT pizza_id, 
      SUM(CASE WHEN exclusion_status = 1 AND extras_status = 1 
              THEN 1 ELSE 0 END) AS pizza_count
      FROM t1 
      WHERE cancel_status = 1
GROUP BY pizza_id 

```

<div>
<img src="../sql-screenshots/pizza_runner_images/s8.png" width="250"/>
</div>

##### 9. What was the total volume of pizzas ordered for each hour of the day?


```sql 
DROP TABLE IF EXISTS t1; 
CREATE TEMP TABLE t1 AS (
SELECT co.order_id, co.order_time, co.pizza_id, DATE_TRUNC('day', co.order_time) AS order_day, 
DATE_TRUNC('hour', co.order_time) AS order_hour
FROM pizza_runner.customer_orders AS co 
LEFT JOIN pizza_runner.runner_orders AS ro 
ON ro.order_id = co.order_id
);

SELECT DATE_PART('HOUR', order_hour) AS order_hour_part, COUNT(pizza_id) FROM t1 
GROUP BY order_hour_part
ORDER BY order_hour_part ASC
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s9.png" width="250"/>
</div>

##### 10. What was the volume of orders for each day of the week?


``` sql 
WITH t1 AS (
  SELECT
	TO_CHAR(order_time, 'Day') AS day,
	COUNT(order_id) AS order_count, 
	DATE_PART('dow', order_time) AS day_of_week 
FROM pizza_runner.customer_orders
GROUP BY order_time
ORDER BY order_time
)

SELECT day, SUM(order_count) AS total_orders
FROM t1 
GROUP BY day_of_week, day
ORDER BY day_of_week 
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s10.png" width="250"/>
</div>

#### B. Runner And Customer Experience 
##### 1. How many runners signed up for each 1 week period? (i.e. week starts 2021-01-01)


```sql
SELECT DATE_TRUNC('week', "registration_date") :: DATE + 4 AS registration_week, 
COUNT(runner_id) AS runner_count
FROM pizza_runner.runners
GROUP BY registration_week
ORDER BY registration_week
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s11.png" width="250"/>
</div>

##### 2. What was the average time in minutes it took for each runner to arrive at the Pizza Runner HQ to pickup the order?

```sql 
WITH t1 AS (
SELECT co.order_id, ro.pickup_time, co.order_time,
   DATE_PART('minute', pickup_time::TIMESTAMP - order_time) AS time_taken
FROM pizza_runner.runner_orders AS ro 
INNER JOIN pizza_runner.customer_orders AS co 
ON ro.order_id = co.order_id 
WHERE pickup_time IS NOT NULL AND pickup_time != 'null' AND nullif(pickup_time, '') IS NOT NULL 
GROUP BY co.order_id,ro.pickup_time, co.order_time
)

SELECT FLOOR(AVG(time_taken)) AS average_time_taken FROM t1
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s12.png" width="250"/>
</div>

##### 3. Is there any relationship between the number of pizzas and how long the order takes to prepare?


```sql 
WITH t1 AS (
SELECT co.order_id, COUNT(co.order_id) AS pizza_count, 
DATE_PART('minute', ro.pickup_time::TIMESTAMP - co.order_time) AS prep_time
FROM pizza_runner.customer_orders AS co 
JOIN pizza_runner.runner_orders AS ro 
ON ro.order_id = co.order_id
WHERE pickup_time != 'null' AND pickup_time IS NOT NULL AND nullif(pickup_time, '') IS NOT NULL 
GROUP BY co.order_id, pickup_time, order_time 
)

SELECT pizza_count, AVG(prep_time) AS average_prep_time FROM t1 
GROUP BY pizza_count
ORDER BY pizza_count  
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s13.png" width="250"/>
</div>

##### 4. What was the average distance travelled for each customer?

```sql 
WITH cte_customer_order_distances AS (
SELECT 
  t1.customer_id,
  t1.order_id, t2.pickup_time, t2.cancellation, t2.duration, t1.order_time,
  REGEXP_REPLACE(t2.distance, '\s*km', '')::NUMERIC AS distance
FROM pizza_runner.customer_orders AS t1
INNER JOIN pizza_runner.runner_orders AS t2
  ON t1.order_id = t2.order_id
WHERE t2.pickup_time != 'null' AND t2.pickup_time IS NOT NULL 
)

SELECT customer_id, ROUND(AVG(distance),1) FROM cte_customer_order_distances
GROUP BY customer_id
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s14.png" width="250"/>
</div>

##### 5. What was the difference between the longest and shortest delivery times for all orders?


```sql 

WITH cte_customer_order_delivery AS (
SELECT 
  t1.order_id, t2.pickup_time, t2.cancellation, t1.order_time,
  REGEXP_REPLACE(duration, 'minutes|\s*mins|\sminute[s]?', '')::NUMERIC AS duration_min
FROM pizza_runner.customer_orders AS t1
INNER JOIN pizza_runner.runner_orders AS t2
  ON t1.order_id = t2.order_id
WHERE t2.pickup_time != 'null' AND t2.pickup_time IS NOT NULL 
)

SELECT MAX(duration_min) - MIN(duration_min) AS duration_diff
FROM cte_customer_order_delivery

```

<div>
<img src="../sql-screenshots/pizza_runner_images/s15.png" width="250"/>
</div>

##### 6. What was the average speed for each runner for each delivery and do you notice any trend for these values?

```sql 
DROP TABLE IF EXISTS ro_edited; 
CREATE TEMP TABLE ro_edited AS (
    SELECT customer_id, pizza_runner.customer_orders.order_id, 
      runner_id, 
      pickup_time, 
      REGEXP_REPLACE(distance, 'km.\s*km|km$', '')::NUMERIC AS distance_km, 
      REGEXP_REPLACE(duration, 'minutes|\s*mins|\sminute[s]?', '')::NUMERIC AS duration_min
FROM pizza_runner.runner_orders
LEFT JOIN pizza_runner.customer_orders
ON pizza_runner.runner_orders.order_id = pizza_runner.customer_orders.order_id
WHERE pickup_time != 'null'
);

WITH t2 AS (
SELECT customer_id, runner_id, DATE_PART('HOUR', pickup_time::TIMESTAMP) AS pickup_hour,
distance_km, duration_min, 
ROUND(distance_km / (duration_min / 60),2) AS speed_km_h 
FROM ro_edited
ORDER BY speed_km_h DESC, runner_id
)

SELECT  runner_id, customer_id, ROUND(AVG(speed_km_h),2) AS avg_speed FROM t2 
GROUP BY  runner_id, customer_id
ORDER BY runner_id, avg_speed DESC
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s16a.png" width="250"/>
</div>

##### 7. What is the successful delivery percentage for each runner?

```sql 
WITH cte AS 
(
	SELECT runner_id, order_id,
          CASE WHEN cancellation is NULL OR cancellation = 'null' OR nullif(cancellation, '') IS NULL 
          THEN 1
          ELSE 0 END AS sucess_delivery
    FROM pizza_runner.runner_orders
)
SELECT runner_id, ROUND( 100*SUM(sucess_delivery)/COUNT(order_id),2) AS success_rate
FROM cte
GROUP BY runner_id
ORDER BY runner_id
```

<div>
<img src="../sql-screenshots/pizza_runner_images/s17.png" width="250"/>
</div>